## Testing Exceptions

Often you need to test that your code **raises** exceptions for invalid inputs.

### The Problem

In [ ]:
def divide(a, b):
    """Divide a by b."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("Both arguments must be numbers")
    return a / b

How do we test that these exceptions are raised correctly?

### Using `pytest.raises()`

In [ ]:
import pytest

def test_divide_by_zero():
    """Test that dividing by zero raises ValueError."""
    with pytest.raises(ValueError):
        divide(10, 0)

def test_divide_invalid_type():
    """Test that invalid types raise TypeError."""
    with pytest.raises(TypeError):
        divide("10", 5)

**How it works:**
- Code inside the `with` block is expected to raise an exception
- ✅ If the exception is raised → test passes
- ❌ If no exception is raised → test fails
- ❌ If a different exception is raised → test fails

### Checking Exception Message

In [ ]:
def test_divide_by_zero_with_message():
    """Test exception is raised with correct message."""
    with pytest.raises(ValueError, match="Cannot divide by zero"):
        divide(10, 0)

def test_divide_invalid_type_with_message():
    """Test TypeError message."""
    with pytest.raises(TypeError, match="Both arguments must be numbers"):
        divide("text", 5)

The `match` parameter accepts a regex pattern. Test passes only if:
1. Correct exception type is raised
2. Exception message matches the pattern

### Practical Example: User Validation

In [ ]:
class ValidationError(Exception):
    """Custom validation error."""
    pass

class User:
    def __init__(self, username, email, age):
        if not username:
            raise ValidationError("Username cannot be empty")
        if "@" not in email:
            raise ValidationError("Invalid email format")
        if age < 0:
            raise ValidationError("Age cannot be negative")
        if age > 150:
            raise ValidationError("Age is unrealistic")
        
        self.username = username
        self.email = email
        self.age = age

def test_user_empty_username():
    """Test empty username raises error."""
    with pytest.raises(ValidationError, match="Username cannot be empty"):
        User("", "alice@example.com", 25)

def test_user_invalid_email():
    """Test invalid email raises error."""
    with pytest.raises(ValidationError, match="Invalid email format"):
        User("alice", "invalid-email", 25)

def test_user_negative_age():
    """Test negative age raises error."""
    with pytest.raises(ValidationError, match="Age cannot be negative"):
        User("alice", "alice@example.com", -5)

def test_user_unrealistic_age():
    """Test unrealistic age raises error."""
    with pytest.raises(ValidationError, match="Age is unrealistic"):
        User("alice", "alice@example.com", 200)

def test_user_valid():
    """Test valid user creation doesn't raise error."""
    user = User("alice", "alice@example.com", 25)
    assert user.username == "alice"
    assert user.email == "alice@example.com"
    assert user.age == 25

---

## Test Markers

**Markers** are tags you can apply to tests to categorize them or control their behavior.

### Built-in Markers

#### `@pytest.mark.skip` - Skip a Test

In [ ]:
@pytest.mark.skip(reason="Feature not implemented yet")
def test_future_feature():
    """This test will be skipped."""
    # Will implement this later
    pass

@pytest.mark.skip(reason="Waiting for API endpoint")
def test_api_integration():
    """Skip until API is ready."""
    pass

**Output:**
```
test_example.py::test_future_feature SKIPPED (Feature not implemented yet)
test_example.py::test_api_integration SKIPPED (Waiting for API endpoint)
```

Tests are skipped and the reason is shown.

#### `@pytest.mark.skipif` - Conditional Skip

In [ ]:
import sys

@pytest.mark.skipif(sys.platform == "win32", reason="Not supported on Windows")
def test_unix_specific():
    """This test only runs on Unix systems."""
    # Unix-specific code
    pass

@pytest.mark.skipif(sys.version_info < (3, 8), reason="Requires Python 3.8+")
def test_modern_python_feature():
    """This test requires Python 3.8 or higher."""
    # Use walrus operator := (Python 3.8+)
    if (n := 10) > 5:
        assert n == 10

**Common use cases:**
- Platform-specific tests (Windows vs Linux vs Mac)
- Python version requirements
- Optional dependencies
- Environment-specific tests (development vs production)

#### `@pytest.mark.xfail` - Expected to Fail

In [ ]:
@pytest.mark.xfail(reason="Known bug #123")
def test_known_bug():
    """This test is expected to fail until bug #123 is fixed."""
    result = buggy_function()
    assert result == expected_value

@pytest.mark.xfail(sys.platform == "win32", reason="Windows bug")
def test_with_windows_bug():
    """Expected to fail on Windows."""
    pass

**Output possibilities:**
- `XFAIL` - Test failed as expected (good!)
- `XPASS` - Test passed unexpectedly (bug might be fixed!)

**Use xfail when:**
- You know a test will fail but want to track it
- There's a known bug that will be fixed later
- You're documenting expected failures

**Development workflow:**
```bash
# Quick feedback during development
pytest -m "unit and fast"  # Run only fast unit tests

# Before committing
pytest -m "unit or smoke"  # Unit tests + smoke tests

# Full test suite
pytest  # Run everything

# CI/CD pipeline
pytest -m "not slow"  # Skip slow tests for faster feedback
pytest -m integration  # Run integration tests separately
```

### Running the Test Suite

**Run all tests:**
```bash
pytest
```

**Run only unit tests (fast):**
```bash
pytest -m unit
```

**Run only integration tests:**
```bash
pytest -m integration
```

**Run with verbose output:**
```bash
pytest -v
```

**Run specific test file:**
```bash
pytest tests/test_user.py
```

**Run specific test class:**
```bash
pytest tests/test_user.py::TestUserValidation
```

**Run specific test:**
```bash
pytest tests/test_user.py::TestUserValidation::test_valid_user
```

---

## Best Practices

### 1. Organize Tests with Markers

**✅ Good - Well organized:**
```python
@pytest.mark.unit
@pytest.mark.fast
def test_addition():
    ...

@pytest.mark.integration
@pytest.mark.slow
@pytest.mark.database
def test_full_workflow():
    ...
```

### 2. Test Exceptions Explicitly

**✅ Good:**
```python
def test_division_by_zero():
    with pytest.raises(ValueError, match="Cannot divide by zero"):
        divide(10, 0)
```

### 3. Use Descriptive Marker Names

**❌ Bad:**
```python
@pytest.mark.test1
@pytest.mark.slow
```

**✅ Good:**
```python
@pytest.mark.integration
@pytest.mark.database
@pytest.mark.slow
```

### 4. Skip vs XFail

**Use `skip` when:**
- Test isn't relevant (platform-specific)
- Feature not implemented yet
- Missing dependencies

**Use `xfail` when:**
- Known bug that will be fixed
- Test documents expected failure
- Want to track failures


---

## Summary

### What We Learned:

✅ **pytest.raises()** - Test that exceptions are raised correctly  
✅ **Markers** - Tag and organize tests  
✅ **@pytest.mark.skip** - Skip tests  
✅ **@pytest.mark.skipif** - Conditional skip  
✅ **@pytest.mark.xfail** - Expected failures  
✅ **Custom markers** - Create your own test categories  
✅ **conftest.py** - Share fixtures across test files  
✅ **Running specific tests** - Use `-m` flag with markers  

### Key Commands:

```bash
pytest -m unit                    # Run unit tests
pytest -m "not slow"             # Skip slow tests
pytest -m "integration and database"  # Multiple markers
pytest -v                        # Verbose output
pytest -k "pattern"              # Run tests matching pattern
```

### Next Steps:

You now have all the essential pytest knowledge! Practice by:
- Writing tests for your own projects
- Organizing tests with markers
- Creating shared fixtures in conftest.py
- Testing edge cases and exceptions

Happy testing! 🎉